<a href="https://colab.research.google.com/github/pathwaycom/pathway-examples/blob/main/documentation/transformer_tree_example.ipynb" target="_parent"><img src="https://pathway.com/assets/colab-badge.svg" alt="Run In Colab" class="inline"/></a>

# [Colab-specific instructions] Installing Pathway with Python 3.8+

> In the cell below we install pathway into a Python 3.8+ Linux runtime.
> 
> **Please run the colab notebook (Ctrl+F9)**, disregarding the 'not authored by Google' warning. **The installation and loading time is less than 1 minute**.


In [ ]:
#@title ⚙️ Pathway installer. Please run to install Pathway.
class InterruptExecution(Exception):
    def _render_traceback_(self):
        pass

DO_INSTALL = False
import sys
if sys.version_info >= (3, 8):
    print(f'✅ Python {sys.version} is active.')
    try:
        import pathway as pw
        print('✅ Pathway successfully imported.')
    except:
        DO_INSTALL = True
else:
    print("⛔ Pathway requires Python 3.8 or higher.")
    raise InterruptExecution

PATHWAY_INSTALL_LINK = 'https://packages.pathway.com/966431ef6ba'
if DO_INSTALL:
    !ls $(dirname $(which python))/../lib/python*/*-packages/pathway 1>/dev/null 2>/dev/null || echo "⌛ Installing Pathway. This usually takes a few seconds..."
    !ls $(dirname $(which python))/../lib/python*/*-packages/pathway 1>/dev/null 2>/dev/null || pip install --extra-index-url {PATHWAY_INSTALL_LINK} pathway 1>/dev/null 2>/dev/null
    !ls $(dirname $(which python))/../lib/python*/*-packages/pathway 1>/dev/null 2>/dev/null || echo "⛔ Installation failed. Don't be shy to reach out to the community at https://pathway.com !"
    !ls $(dirname $(which python))/../lib/python*/*-packages/pathway 1>/dev/null 2>/dev/null && echo "✅ All installed. Enjoy Pathway!"


# Implementing a tree using transformer classes
A more advanced example on how transformer classes work by implementing a tree.

Pathway's transformer class is a powerful tool to perform advanced operations on tables.
In the following, we are going to show you how to use a transformer class to implement a tree and compute recursive operations on it.

We strongly advise you to read our [introduction on transformer classes](https://pathway.com/developers/documentation/transformer-classes/transformer-intro) and the [simple examples](https://pathway.com/developers/documentation/transformer-classes/transformer-example) before reading further.

## Pathway Data Structures \& Algorithms 101: How to represent a Tree?

Let's take a look at one of the simplest graph-like data structures: a tree. Let's encode tree nodes into a table with columns:

1. Node ID
2. A value `val` of integer type, stored in nodes of the tree.
3. The node's parent ID - which can be Null for the root.

To do this, in Pathway you can write the following schema for the considered table (ID's are implicit and don't need to be defined).

In [1]:
from __future__ import annotations

from typing import Optional

import pathway as pw


class Nodes(pw.Schema):
    val: int
    parent: Optional[Nodes]

## Transformer Classes acting on a single row

You would now like to compute some basic statistics on the tree. For example, is a given node the root? In Python, this would follow through a simple row operation:

```py
    # We would want to add this logic as a "method" to the `nodes` schema

    def is_root(self):
        return self.parent is None
```

How to make a transformer which takes a table following the schema `nodes` and "gives it" the method above? The answer is a Transformer Class which acts on a single table argument called `nodes`, and adds the `is_root` logic as an output argument. We call our transformer `tree_node_roots`:

In [2]:
@pw.transformer
class tree_node_roots:
    class nodes(pw.ClassArg, input=Nodes):
        val = pw.input_attribute()
        parent = pw.input_attribute()

        @pw.output_attribute
        def is_root(self):
            return self.parent is None

Let's provide a quick explanation of what happens here.
You can specify `Nodes` as input for the class `nodes` to enforce that the rows of the table are of type `Nodes`.
You link the parameters of the class `nodes` to the ones of `Nodes` with the `pw.input_attribute()` function. Note that the names of the parameters (`val` and `parent` in the example) must be exactly the same as the column names of the input table.
Finally, you declare the different columns of the resulting table using the annotation `pw.output_attribute` on different functions. Each function defines a column in the output table and the value of the function is going to be used to as the value: the name of the function defines the name of the column.

You can now use `tree_node_roots` as a transformer, and call `tree_node_roots(TN)` for a table `TN` of nodes to get the required output columns, just as you would for any other transformer.

In [3]:
tree = pw.debug.table_from_markdown(
    """
    | val | parent_label
 0  | 0    |
 1  | 1    | 0
 2  | 2    | 0
 3  | 3    | 1
 4  | 4    | 1
 5  | 5    | 2
 6  | 6    | 2
 """
)
tree += tree.select(parent=tree.pointer_from(tree.parent_label, optional=True))
pw.debug.compute_and_print(tree)

result = tree_node_roots(tree).nodes
pw.debug.compute_and_print(result)

            | val | parent_label | parent
^8JFNKVV... | 0   |              |
^2TMTFGY... | 1   | 0            | ^8JFNKVV...
^YHZBTNY... | 2   | 0            | ^8JFNKVV...
^SERVYWW... | 3   | 1            | ^2TMTFGY...
^8GR6BSX... | 4   | 1            | ^2TMTFGY...
^76QPWK3... | 5   | 2            | ^YHZBTNY...
^C4S6S48... | 6   | 2            | ^YHZBTNY...
            | is_root
^YHZBTNY... | False
^2TMTFGY... | False
^76QPWK3... | False
^8GR6BSX... | False
^SERVYWW... | False
^C4S6S48... | False
^8JFNKVV... | True


## Transformer Classes acting on multiple rows

Now, let's try something which shows the power of Pathway a bit more. Suppose you would like to see how many steps away a node is from its root. Let's call this the `level` of a node. How would you compute this?

Logically, the `level` of a node is higher by 1 unit than the `level` of its parent. So, the solution can be obtained by recursion.

Recursion is perhaps something you would think twice about before [attempting in SQL](https://medium.com/swlh/recursion-in-sql-explained-graphically-679f6a0f143b). In Pathway, recursion is natively supported, and efficient to use where the "recursion stack" does not change much for old data rows as new data arrives.

The transformer which does just what we want is provided below.

In [4]:
@pw.transformer
class tree_node_roots_and_levels:
    class nodes(pw.ClassArg, input=Nodes):
        val = pw.input_attribute()
        parent = pw.input_attribute()

        @pw.output_attribute
        def is_root(self):
            return self.parent is None

        @pw.output_attribute
        def level(self):
            if self.is_root:
                return 0
            else:
                return 1 + self.transformer.nodes[self.parent].level

Most of the logic is contained in the final line, `1 + self.transformer.nodes[self.parent].level`.

You obtain the following table:

In [5]:
result = tree_node_roots_and_levels(tree).nodes
pw.debug.compute_and_print(result)

            | is_root | level
^YHZBTNY... | False   | 1
^2TMTFGY... | False   | 1
^76QPWK3... | False   | 2
^8GR6BSX... | False   | 2
^SERVYWW... | False   | 2
^C4S6S48... | False   | 2
^8JFNKVV... | True    | 0


A small side note: you might simply have wanted to write here `1 + self.parent.level` instead, however, this would be missing information about the table that `self.parent` lives in. This table is identified through `self.transformer.nodes`.

Though making the syntax a bit more verbose, identifying objects through both a table, and a row identifier, helps to avoid confusion.

You will see why this is useful in this [article](https://pathway.com/developers/documentation/transformer-classes/transformer-example) where we introduce Transformer Classes that use not just one, but two or more arguments. These will allow us to work with a `matchings` table and a `profiles` table, indicating a pair of nodes for which the required computation should be performed.


## Conclusion

In this guide, you learned how to write transformer classes building a tree and computing some basic operations on that tree. This is useful for defining row-based logic for tables, oblivious of the fact that we are operating on top of data streams.
You can take a look at our [tour of Pathway's transformers](https://pathway.com/developers/documentation/transformer-classes/transformer-example) in which you will find a list of examples of transformers.

You can also check our [connectors](https://pathway.com/developers/documentation/input-and-output-streams/connectors) to connect your data into Pathway.